In [107]:
class PriorityQ(object):
    def __init__(self, lst=[]):
        self.lst = lst
        self.heap = self.build_heap()

    def parent(self,i):
        return (i-1)//2
    def left_child(self,i):
        return 2*i+1
    def right_child(self,i):
        return 2*i+2
    
    def insert(self,val):
        self.heap.append(val)
        self.sift_up(len(self.heap)-1,self.heap)
    def extractMax(self):
        self.heap[0],self.heap[-1] = self.heap[-1],self.heap[0]
        result = self.heap.pop()
        self.sift_down(0,self.heap)
        return result
    def change_priority(i,p):
        pass

    def sift_up(self,i,H):
        while i>1 and H[self.parent(i)] < H[i]:
            H[self.parent(i)],H[i] = H[i],H[self.parent(i)]
            i = self.parent(i)

    def sift_down(self,i,H):
        max_idx = i
        try:
            l = self.left_child(i)
            if l <= len(H) and H[l] > H[max_idx]:
                max_idx = l
        except:
            pass
        try:
            r = self.right_child(i)
            if r <= len(H) and H[r] > H[max_idx]:
                max_idx = r
        except:
            pass
        if i != max_idx:
            H[i],H[max_idx] = H[max_idx],H[i]
            print("swap: ", i,max_idx)
            print( "heap", H)
            self.sift_down(max_idx,H)

    def build_heap(self):
        n = len(self.lst)
        H = self.lst
        for i in reversed(range(n//2)):
            self.sift_down(i,H)
        return H
    def print_heap(self):
        print(self.heap)

In [108]:
H = [5, 4, 3, 2, 1]
K = [1, 2, 3, 5, 4]
p = PriorityQ(K)
p.print_heap()

swap:  1 3
heap [1, 5, 3, 2, 4]
swap:  0 1
heap [5, 1, 3, 2, 4]
swap:  1 4
heap [5, 4, 3, 2, 1]
[5, 4, 3, 2, 1]


In [109]:
p.insert(-1)
p.print_heap()

[5, 4, 3, 2, 1, -1]


In [110]:
p.insert(9)

In [111]:
p.print_heap()

[9, 4, 5, 2, 1, -1, 3]


In [112]:
v=p.extractMax()
p.print_heap()

swap:  0 2
heap [5, 4, 3, 2, 1, -1]
[5, 4, 3, 2, 1, -1]
